# &#x1F4D1; &nbsp; <span style="color:red"> Reflections. Deep Learning. Lesson 3</span>

##   &#x1F916; &nbsp; <span style="color:red">Links</span>

CS231n Convolutional Neural Networks for Visual Recognition

- Optimization http://cs231n.github.io/optimization-1/
- Intuitive understanding of backpropagation http://cs231n.github.io/optimization-2/
- Neural Network architectures http://cs231n.github.io/neural-networks-1/
- Data Preprocessing, Loss Functions http://cs231n.github.io/neural-networks-2/
- Gradient Check, Sanity Check http://cs231n.github.io/neural-networks-3/
- Convolutional Neural Networks http://cs231n.github.io/convolutional-networks/
- Visualizing Convolutional Neural Networks http://cs231n.github.io/understanding-cnn/
- Transfer Learning http://cs231n.github.io/transfer-learning/

Convolutional Neural Networks (LeNet) http://deeplearning.net/tutorial/lenet.html
    
CS231n Winter 2016: Lecture 7: Convolutional Neural Networks https://www.youtube.com/watch?v=LxfUGhug-iQ

Some Lesser-Known Deep Learning Libraries
http://blog.paralleldots.com/technology/deep-learning/lesser-known-deep-learning-libraries/?utm_content=buffer02038&utm_medium=social&utm_source=facebook.com&utm_campaign=buffer

##   &#x1F916; &nbsp; <span style="color:red">Libraries</span>

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from IPython.display import display, Image

##   &#x1F916; &nbsp; <span style="color:red">CNN Convolutional Neural Networks</span>

Convolutional Neural Networks have extraordinary good performance on image classification tasks. The key idea behind CNNs is to build many layers of feature detectors to take the spatial arrangement of pixels in an input image into account.

##   &#x1F916; &nbsp; <span style="color:red">Assignment 4: Convolutional Models</span>

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [4]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], 
                                                     stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.058804
Minibatch accuracy: 18.8%
Validation accuracy: 9.7%
Minibatch loss at step 50: 2.003699
Minibatch accuracy: 25.0%
Validation accuracy: 33.6%
Minibatch loss at step 100: 1.280552
Minibatch accuracy: 56.2%
Validation accuracy: 70.9%
Minibatch loss at step 150: 0.541837
Minibatch accuracy: 93.8%
Validation accuracy: 74.3%
Minibatch loss at step 200: 1.185073
Minibatch accuracy: 62.5%
Validation accuracy: 76.5%
Minibatch loss at step 250: 1.194932
Minibatch accuracy: 68.8%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.570196
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.578284
Minibatch accuracy: 93.8%
Validation accuracy: 78.0%
Minibatch loss at step 400: 0.236324
Minibatch accuracy: 93.8%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.845133
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.713367
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Mi

Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.
